In [ ]:
import gzip
import string
import pandas as pd
import numpy as np
from collections import defaultdict
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
def readGz(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [ ]:
userItems = []

for d in readGz("australian_users_items.json.gz"):
    userItems.append(d)


In [ ]:
features = []
for d in userItems:
    for i in d['items']:
        features.append((d['user_id'], i['item_name'], i['playtime_forever']))

In [ ]:
itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)

for f in features:
    u = f[0]
    i = f[1]
    t = f[2]
    if t > 0:
        itemsPerUser[u].add(i)
        usersPerItem[i].add(u)

In [ ]:
df = pd.DataFrame(features, columns = ['User', 'Game', 'Playtime(min)'])
df = df.reset_index(drop=True)
df['Played'] = df['Playtime(min)'] > 0
df = df.drop('Playtime(min)', axis=1)
df.head(10)

In [ ]:
len(df)

In [ ]:
#Train-validation split
train = df.iloc[:200000]
validation = df.iloc[200000:300000]

In [ ]:
    gameCount = defaultdict(int)
    totalPlayed = 0

    for x in train.itertuples():
        gameCount[x.Game] += 1
        totalPlayed += 1

    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()

In [ ]:
def baseline(train, valid, t=1.5):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPlayed * t: break
        
    pred = []
    for x in valid.itertuples():
        if (x.Game in return1):
            pred.append(True)
        else:
            pred.append(False)
    valid['predB'] = pred
    accuracy = (valid.predB == valid['Played']).sum()/len(valid['Played'])
    return accuracy

In [ ]:
baseline(train, validation, 0.6)

In [ ]:
X = [i/100 for i in range(50,105) ]
y = []
best_acc = 0
for t in range(50,105):
    acc = baseline(train,validation, (t / 100))
    y.append(acc)
    if acc > best_acc:
        best_acc = acc
        best_t = t

plt.plot(X,y)
print('Best Threshold:', best_t/100)
print('Best Accuracy:', best_acc)
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.title('Threshold VS Accuracy')
plt.show()

In [ ]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
def jac_pred(train, valid, t=0.05, t2 = 0.8):
    itemsPerUserTrain = defaultdict(set)
    usersPeritemTrain = defaultdict(set)

    for x in train.itertuples():
        if x.Played:
            itemsPerUserTrain[x.User].add(x.Game)
            usersPeritemTrain[x.Game].add(x.User)

    pred = []
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPlayed * t2: break

    for x in valid.itertuples():
        maxJacc = 0
        
        # finding max jacc sim based on users
        for game in itemsPerUser[x.User]:
            if x.Game == game:
                continue
            jacc = Jaccard(usersPeritemTrain[x.Game], usersPeritemTrain[game])
            if jacc > maxJacc:
                maxJacc = jacc
                if jacc > t:
                    break
                    
        if ((maxJacc > t) and (x.Game in return1)):
            pred.append(True)
        else:
            pred.append(False)
    valid['predJ'] = pred
    return (valid.predJ == valid.Played).sum()/len(valid.Played)

In [ ]:
jac_pred(train,validation)

In [ ]:
def Euclidean(s1,s2):
    return len(s1.union(s2)) - len(s1.intersection(s2))